In [1]:
import pandas as pd
import numpy as np


import matplotlib.pylab as plt
import seaborn as sns
sns.set_style('whitegrid')

from collections import defaultdict
from progressbar import ProgressBar
import re
from multiprocessing import Pool
import time

import scipy
import networkx as nx
import re

import os
import random
import statsmodels.api as sm
from collections import Counter

from statsmodels.stats.multitest import multipletests

import sys

sys.path.append('../')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
basedir = os.path.abspath('../') + '/'
infolder = basedir + 'data/'
outfolder = basedir + 'output/'
dbs = basedir + 'data/databases/'

In [3]:
import utils.network_utils as network_utils

In [4]:
infile = infolder + 'HumanInteractome_v2017.csv'
header=True
sep = ','
columns = ['EntrezA', 'EntrezB']
lcc = False
G = network_utils.parse_interactome(infile, sep, header, columns, lcc=True)

print (len(G.nodes()), 'nodes')
print (len(G.edges()), 'edges')

17651 nodes
351393 edges


In [5]:
polyphenol = pd.read_csv(infolder + 'PolyphenolProteinInteractions.csv')
chemical2genes = defaultdict(list)
for i in polyphenol.index:
    name = polyphenol.chemical.loc[i]
    chemical2genes[name].append(polyphenol.entrez_id.loc[i])
    
for i in chemical2genes.keys():
    chemical2genes[i] = list(set(chemical2genes[i]))

In [11]:
dg = pd.read_csv(infolder + 'GenesDisease.csv')
disease2genes = defaultdict(list)

for i in dg.index:
    disease2genes[dg.disease.loc[i]].append(dg.entrez_id.loc[i])

## Import GCT

* Perturbation signatures (in z-scores) of MCF7 cells treated with polyphenols were downloaded from the CLUE.io website (https://clue.io/)

In [12]:
def parse_gct(infile):
    
    dt = pd.read_table(infile, sep = '\t', skiprows=2)
    cols = list(dt.columns[9:])
    desc = dt.loc[:7, cols]
    cols.append('id')
    exp_matrix = dt[cols]
    exp_matrix = dt.iloc[8:]
    attributes = defaultdict(dict)
    desc = dt.loc[:7, cols]
    for i in desc.columns[:-1]:
        for j in desc.index:
            attributes[i][desc['id'].loc[j]] = desc[i].loc[j]
    attributes = pd.DataFrame.from_dict(attributes, orient='index').reset_index()
    attributes['distil_cc_q75'] = attributes['distil_cc_q75'].astype(float)
    return(exp_matrix, attributes)


def get_sp(gene):
    if gene in G.nodes():
        s = network_utils.calculate_closest_distance(G,[gene], genes)
    else:
        s = float('nan')
    t = (gene,s)
    return(t)

In [13]:
def get_gsea(geneset, nongeneset, df,dataframe=False):
    dic = {}
    N_H = len(geneset)
    N = df.shape[0]
    n_r = np.sum(df['value'][df.gene.isin(geneset)].abs())
    for rank in range(N):
        dx = df.iloc[:rank]
        s = list(set(geneset) & (set(dx['gene'])))
        p_hit = np.sum(dx[dx.gene.isin(s)]['value'].abs()/n_r)
        p_miss = 1.0/(N - N_H) * dx[~dx.gene.isin(geneset)].shape[0]
        dic[rank] = p_hit - p_miss
    dic = pd.DataFrame.from_dict(dic, orient='index')
    if dataframe:
        return(dic)
    else:
        return(dic[0].abs().max())
    
    
def get_gsea_performance(geneset, nongeneset, df):
    df = df.reset_index()
    ranks_mapped = list(df[df.gene.isin(geneset)].index)
    ranks = []
    for i in ranks_mapped:
        ranks.append(i - 1)
        ranks.append(i)
        ranks.append(i + 1)
    dic = {}
    N_H = len(geneset)
    N = df.shape[0]
    n_r = np.sum(df['value'][df.gene.isin(geneset)].abs())
    for rank in ranks:
        dx = df.iloc[:rank]
        s = list(set(geneset) & (set(dx['gene'])))
        p_hit = np.sum(dx[dx.gene.isin(s)]['value'].abs()/n_r)
        p_miss = 1.0/(N - N_H) * dx[~dx.gene.isin(geneset)].shape[0]
        dic[rank] = p_hit - p_miss
    dic = pd.DataFrame.from_dict(dic, orient='index')
    return(dic[0].abs().max())


def get_null(input):
    n, df = input
    geneset = df['gene'].sample(n)
    nongeneset = list(set(df['gene']) - set(geneset))
    #es = get_gsea(geneset, nongeneset, df)
    es = get_gsea_performance(geneset, nongeneset, df)
    return(es)

In [14]:
molecules = ['coumarin.gct',
 'nobiletin.gct',
 'prunetin.gct',
 'coumestrol.gct',
 'pterostilbene.gct',
 'isoliquiritigenin.gct',
 'caffeic_acid.gct',
 'apigenin.gct',
 'naringenin.gct',
 '(-)-epicatechin 3-o-gallate.gct',
 '(-)-epigallocatechin 3-o-gallate.gct',
 'naringin.gct',
 'piceatannol.gct',
 'epicatechin.gct',
 'daidzein.gct',
 'umbelliferone.gct',
 'rosmarinic acid.gtc']

In [15]:
infiles = [i for i in os.listdir(basedir + 'data/connectivity_map/') if i.endswith('.gct')]

In [30]:
ncpu = 1
npermutations = 10
for infile in infiles:
    
    cell_line = 'MCF7'
    
    ## parse gct file
    dt, attributes = parse_gct(basedir + 'data/connectivity_map/' + infile)
    
    ## select experimental instances to run gsea
    selected = attributes[(attributes.cell_id == cell_line)]
    timepoints = ['24 h', '6 h']
    treats = list(set(attributes['pert_idose']))
    samples = []
    for tp in timepoints:
        for treat in treats:
            buf = selected[(selected['pert_itime'] == tp) & (selected['pert_idose'] == treat)]
            if buf.shape[0] == 1:
                samples.append(buf['index'].iloc[0])
            if buf.shape[0] > 1:
                sample = buf[buf['distil_cc_q75'] == buf['distil_cc_q75'].max()]['index'].iloc[0]
                samples.append(sample)
    
    
    
    sample2attributes = {}
    for sample in samples:
        dic = {}
        buf = attributes[attributes['index'] == sample][['cell_id', 'name', 'pert_idose', 'pert_itime', 'index']]
        sample2attributes[sample] = buf.to_dict('records')[0]
                
    
    ## prepare tables
    cols = ['id'] + samples
    data = dt[cols]
    data['id'] = data['id'].astype(int)
    data = data[data['id'].isin(list(G.nodes()))]
    for col in samples:
        data[col] = data[col].astype(float)
    
    
    npermutations = 10
    
    ## run gsea for each experimental instance
    for test in samples:
        
        dquery = data[[test, 'id']]
        dquery = dquery.sort_values(by = test)
        dquery.columns = ['value', 'gene']
        
        dfs = []
        pbar = ProgressBar()
        dic_null = {}
        for disease in pbar(disease2genes.keys()):
            res = defaultdict(dict)
            c = 0
            geneset = list(set(disease2genes[disease]) & set(G.nodes()))
            ns = list(set(dquery['gene']) - set(geneset))
            es = get_gsea_performance(geneset, ns, dquery)
            if not len(geneset) in dic_null.keys():
                p = Pool(ncpu)
                randomsets = np.repeat(len(geneset),npermutations)
                randomsets = [[i,dquery] for i in randomsets]
                null = p.map(get_null, randomsets)
                p.close()
                dic_null[len(geneset)] = null
            res[c]['disease'] = disease
            res[c]['ES'] = es
            res[c]['Z_ES'] = (es - np.mean(dic_null[len(geneset)]))/np.std(dic_null[len(geneset)])
            res[c]['p_empirical'] = len([i for i in dic_null[len(geneset)] if i > es])/len(dic_null[len(geneset)])
            for att in sample2attributes[sample]:
                res[c][att] = sample2attributes[sample][att]
            res = pd.DataFrame.from_dict(res, orient='index')
            #res.to_csv(outfolder + 'tables/tmp/%s.csv'%disease)
            dfs.append(res)
        res = pd.concat(dfs)
    
    break

/Users/italodovalle/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (0,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/italodovalle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Process ForkPoolWorker-15:                                                    |
Traceback (most recent call last):
  File "/Users/italodovalle/anaconda3/lib/python3.7/multiprocessing/process.py"

KeyboardInterrupt: 

In [33]:
dfs[0]

,disease,ES,Z_ES,p_empirical,cell_id,name,pert_idose,pert_itime,index
0,kidney diseases,0.225846,1.392548,0.0,MCF7,isoliquiritigenin,10 µM,6 h,CPC006_MCF7_6H:BRD-K33583600-001-09-6:10


[{'cell_id': 'MCF7',
  'name': 'isoliquiritigenin',
  'pert_idose': '10 µM',
  'pert_itime': '24 h',
  'index': 'CPC006_MCF7_24H:BRD-K33583600-001-09-6:10'}]

In [23]:
attributes[attributes['index'] == sample]

,index,cell_id,pert_id,name,pert_type,pert_idose,pert_itime,distil_cc_q75,distil_ss
25,CPC006_MCF7_24H:BRD-K33583600-001-09-6:10,MCF7,BRD-K33583600,isoliquiritigenin,trt_cp,10 µM,24 h,0.59,3.6809


## Organize Output

In [ ]:
molecule = 'genistein'
cell_line = 'MCF7'
indir = '/home/italodovalle/flavonoids/output/tables/tmp/'
dfs = []
for i in os.listdir(indir):
    if 'gsea_%s_%s'%(molecule, cell_line.lower()) in i:
        d = pd.read_csv(indir + i, index_col = 0)
        if '_' in molecule:
            v = i.split('_')
            cell, concentration, time = v[-3],v[-2],v[-1]
            chemical = molecule
        else:
            buf, chemical, cell,concentration, time =  i.split('_')
        d['cell'] = cell
        d['concentration'] = concentration
        d['duration'] = time.split('.')[0]
        d['pvalue_adj'] = multipletests(d['p_empirical'], alpha=0.05, method='fdr_bh')[1]
        dfs.append(d)
df = pd.concat(dfs)
df.to_csv(outfolder + 'tables/gsea_cmap/%s_%s_cmap_gsea.csv'%(molecule,cell_line.lower()))